In [3]:
!pip install -q gradio
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


ValueError: mount failed

In [4]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import io

# ✅ Only accept images with ≥10% green-dominant pixels
def is_leaf(image):
    img = image.convert("RGB")
    img_np = np.array(img)
    r, g, b = img_np[:, :, 0], img_np[:, :, 1], img_np[:, :, 2]
    green_mask = (g > r * 1.2) & (g > b * 1.2)
    green_ratio = np.sum(green_mask) / (img_np.shape[0] * img_np.shape[1])
    return green_ratio >= 0.10

# ✅ Heuristic crop trait check (color + aspect ratio)
crop_traits = {
    "Apple": {"green_range": (90, 180), "aspect_ratio": (0.8, 1.3)},
    "Cherry": {"green_range": (60, 130), "aspect_ratio": (1.1, 1.6)},
    "Tomato": {"green_range": (100, 200), "aspect_ratio": (0.6, 1.2)},
    "Corn": {"green_range": (80, 170), "aspect_ratio": (1.5, 2.5)},
    "Potato": {"green_range": (90, 170), "aspect_ratio": (0.8, 1.4)},
    "Peach": {"green_range": (80, 150), "aspect_ratio": (0.9, 1.5)},
    "Strawberry": {"green_range": (100, 190), "aspect_ratio": (0.7, 1.3)},
    "Bell Pepper": {"green_range": (80, 160), "aspect_ratio": (0.9, 1.4)},
    "Grape": {"green_range": (70, 140), "aspect_ratio": (0.7, 1.2)},
}

def extract_leaf_traits(image):
    img = image.convert("RGB")
    img_np = np.array(img)
    green = img_np[:, :, 1].astype(float)
    avg_green = np.mean(green)
    aspect_ratio = img_np.shape[1] / img_np.shape[0]
    return {"green": avg_green, "aspect_ratio": aspect_ratio}

# ✅ Disease label mappings (crop-specific)
disease_names = {
    'Apple': {0: 'Apple Scab', 1: 'Black Rot', 2: 'Cedar Apple Rust', 3: 'Healthy'},
    'Bell Pepper': {0: 'Bacterial Spot', 1: 'Healthy'},
    'Cherry': {0: 'Healthy', 1: 'Powdery Mildew'},
    'Corn': {0: 'Cercospora Leaf Spot', 1: 'Common Rust', 2: 'Healthy', 3: 'Northen Leaf Blight'},
    'Grape': {0: 'Black Rot', 1: 'Esca (Black Measles)', 2: 'Healthy', 3: 'Leaf Blight'},
    'Peach': {0: 'Bacterial Spot', 1: 'Healthy'},
    'Potato': {0: 'Early Blight', 1: 'Healthy', 2: 'Late Blight'},
    'Strawberry': {0: 'Healthy', 1: 'Leaf Scorch'},
    'Tomato': {
        0: 'Bacterial Spot',
        1: 'Early Blight',
        2: 'Healthy',
        3: 'Late Blight',
        4: 'Seporia Leaf Spot',
        5: 'Yellow Leaf Curl Virus'
    }
}

# ✅ Paths to your trained crop-specific models
model_paths = {
    "Apple": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_apple.h5",
    "Bell Pepper": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_bell_paper.h5",
    "Cherry": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_cherry.h5",
    "Corn": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_corn.h5",
    "Grape": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_grape.h5",
    "Peach": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_peach.h5",
    "Potato": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_potato.h5",
    "Strawberry": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_strawberry.h5",
    "Tomato": "/content/drive/MyDrive/plant_disease_detection_models/plant_disease_model_tomato.h5"
}

# 🔮 Prediction logic
def predict(crop, image):
    if not is_leaf(image):
        return "❌ Not a valid leaf image. Please upload a real leaf.", image, None

    traits = extract_leaf_traits(image)
    expected = crop_traits.get(crop)

    if expected:
        green_ok = expected["green_range"][0] <= traits["green"] <= expected["green_range"][1]
        ratio_ok = expected["aspect_ratio"][0] <= traits["aspect_ratio"] <= expected["aspect_ratio"][1]

        if not (green_ok and ratio_ok):
            return f"⚠️ This doesn't appear to be a {crop} leaf (based on color/shape).", image, None

    # Load model and predict
    model = tf.keras.models.load_model(model_paths[crop])
    class_index_to_label = disease_names[crop]
    class_names = [class_index_to_label[i] for i in sorted(class_index_to_label)]

    image_resized = image.convert("RGB").resize((256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(image_resized)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)[0]
    predicted_index = int(np.argmax(predictions))
    predicted_label = class_index_to_label[predicted_index]
    confidence = predictions[predicted_index] * 100

    # Plot probabilities
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(class_names, predictions, color="mediumseagreen")
    ax.set_ylabel("Probability")
    ax.set_xticks(range(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha="right")
    ax.set_title("Prediction Probabilities")
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    chart_img = Image.open(buf)

    return f"Disease: {predicted_label} ({confidence:.2f}%)", image, chart_img

# 🌱 Launch Gradio Interface
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Dropdown(choices=list(model_paths.keys()), label="Select Crop"),
        gr.Image(type="pil", label="Upload Leaf Image")
    ],
    outputs=[
        gr.Label(label="Prediction"),
        gr.Image(label="Uploaded Image"),
        gr.Image(label="Prediction Chart")
    ],
    title="🌿 Plant Disease Detection",
    description="Upload a leaf image and select its crop. The app only predicts for valid leaf images from the correct crop.",
    css="""
        body { background-color: #e8f5e9; }
        .gr-button { background-color: #4caf50 !important; color: white; }
        .gr-box, .gr-panel { border-color: #81c784 !important; }
    """
)

interface.launch(debug=True)
